In [45]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn

device = torch.device('cuda:6' if torch.cuda.is_available() else 'cpu')

In [46]:

# 读取../data/train_data.csv文件
train_df = pd.read_csv('../data/train_data.csv')
# val_df = pd.read_csv('../data/test_data.csv')


train_df['dteday'] = pd.to_datetime(train_df['dteday'])

# # 提取年、月、日，添加为新的整数字段
# train_df['year'] = train_df['dteday'].dt.year
# train_df['month'] = train_df['dteday'].dt.month
# train_df['day'] = train_df['dteday'].dt.day


# val_df['dteday'] = pd.to_datetime(val_df['dteday'])

# # 提取年、月、日，添加为新的整数字段
# val_df['year'] = val_df['dteday'].dt.year
# val_df['month'] = val_df['dteday'].dt.month
# val_df['day'] = val_df['dteday'].dt.day

# 删除date列
# train_df.drop('dteday', axis=1, inplace=True)
# val_df.drop('dteday', axis=1, inplace=True)


# 查看数据集的前几行，并打印数据集的基本信息

print(train_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15219 entries, 0 to 15218
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   instant     15216 non-null  float64       
 1   dteday      15216 non-null  datetime64[ns]
 2   season      15216 non-null  float64       
 3   yr          15216 non-null  float64       
 4   mnth        15216 non-null  float64       
 5   hr          15216 non-null  float64       
 6   holiday     15216 non-null  float64       
 7   weekday     15216 non-null  float64       
 8   workingday  15216 non-null  float64       
 9   weathersit  15216 non-null  float64       
 10  temp        15216 non-null  float64       
 11  atemp       15216 non-null  float64       
 12  hum         15216 non-null  float64       
 13  windspeed   15216 non-null  float64       
 14  casual      15216 non-null  float64       
 15  registered  15216 non-null  float64       
 16  cnt         15216 non-

In [47]:
train_df.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1.0,2011-01-01,1.0,0.0,1.0,0.0,0.0,6.0,0.0,1.0,0.24,0.2879,0.81,0.0,3.0,13.0,16.0
1,2.0,2011-01-01,1.0,0.0,1.0,1.0,0.0,6.0,0.0,1.0,0.22,0.2727,0.80,0.0,8.0,32.0,40.0
2,3.0,2011-01-01,1.0,0.0,1.0,2.0,0.0,6.0,0.0,1.0,0.22,0.2727,0.80,0.0,5.0,27.0,32.0
3,4.0,2011-01-01,1.0,0.0,1.0,3.0,0.0,6.0,0.0,1.0,0.24,0.2879,0.75,0.0,3.0,10.0,13.0
4,5.0,2011-01-01,1.0,0.0,1.0,4.0,0.0,6.0,0.0,1.0,0.24,0.2879,0.75,0.0,0.0,1.0,1.0


In [48]:


categorical_covariates = ['season','yr','mnth','hr','holiday','weekday','workingday','weathersit']

categorical_covariates_num_embeddings = []
for col in categorical_covariates:
    categorical_covariates_num_embeddings.append(train_df[col].nunique())
    # categorical_covariates_num_embeddings[1].append(val_df[col].nunique())

categorical_static = ['instant']

categorical_static_num_embeddings = []
for col in categorical_static:
    train_df[col] = train_df[col].astype('category').cat.codes
    categorical_static_num_embeddings.append(train_df[col].nunique())


numeric_covariates = ['temp','atemp','hum','windspeed','casual','registered','cnt']
target_idx = np.where(np.array(numeric_covariates)=='cnt')[0][0]



# ,,,,,,,,,

In [49]:
train_df.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,0,2011-01-01,1.0,0.0,1.0,0.0,0.0,6.0,0.0,1.0,0.24,0.2879,0.81,0.0,3.0,13.0,16.0
1,1,2011-01-01,1.0,0.0,1.0,1.0,0.0,6.0,0.0,1.0,0.22,0.2727,0.80,0.0,8.0,32.0,40.0
2,2,2011-01-01,1.0,0.0,1.0,2.0,0.0,6.0,0.0,1.0,0.22,0.2727,0.80,0.0,5.0,27.0,32.0
3,3,2011-01-01,1.0,0.0,1.0,3.0,0.0,6.0,0.0,1.0,0.24,0.2879,0.75,0.0,3.0,10.0,13.0
4,4,2011-01-01,1.0,0.0,1.0,4.0,0.0,6.0,0.0,1.0,0.24,0.2879,0.75,0.0,0.0,1.0,1.0


In [50]:
# def dataframe_to_tensor(series,numeric_covariates,categorical_covariates,target_idx):

#     numeric_cov_arr = np.array([series[numeric_covariates].values.tolist()])
#     category_cov_arr = np.array([series[categorical_covariates].values.tolist()])
#     static_cov_arr = np.array(series[categorical_static].values.tolist())

#     x_numeric = torch.tensor(numeric_cov_arr,dtype=torch.float32).transpose(2,1)
#     x_numeric = torch.log(x_numeric+1e-5)
#     x_category = torch.tensor(category_cov_arr,dtype=torch.long).transpose(2,1)
#     x_static = torch.tensor(static_cov_arr,dtype=torch.long)
#     y = torch.tensor(numeric_cov_arr[:,target_idx,:],dtype=torch.float32)

#     return x_numeric, x_category, y


# window_size = 16
# forecast_length = 16
# num_val = 2

# val_max_date = '2012/10/1'
# train_max_date = str((pd.to_datetime(val_max_date) - pd.Timedelta(days=window_size*num_val+forecast_length)).date())

# train_final = train_df[train_df['dteday']<=train_max_date]
# val_final = train_df[(train_df['dteday']>train_max_date)&(train_df['dteday']<=val_max_date)]
# train_final = train_df
# # val_final = val_df

# train_series = train_final.groupby(categorical_static+['family']).agg(list).reset_index()
# # val_series = val_final.groupby(categorical_static+['family']).agg(list).reset_index()

# x_numeric_train_tensor, x_category_train_tensor, y_train_tensor = dataframe_to_tensor(train_df,numeric_covariates,categorical_covariates,target_idx)

# # x_numeric_val_tensor, x_category_val_tensor, y_val_tensor = dataframe_to_tensor(val_df,numeric_covariates,categorical_covariates,target_idx)

In [51]:
def dataframe_to_tensor(series,numeric_covariates,categorical_covariates,categorical_static,target_idx):

    numeric_cov_arr = np.array(series[numeric_covariates].values.tolist())
    category_cov_arr = np.array(series[categorical_covariates].values.tolist())
    static_cov_arr = np.array(series[categorical_static].values.tolist())

    x_numeric = torch.tensor(numeric_cov_arr,dtype=torch.float32).transpose(2,1)
    x_numeric = torch.log(x_numeric+1e-5)
    x_category = torch.tensor(category_cov_arr,dtype=torch.long).transpose(2,1)
    x_static = torch.tensor(static_cov_arr,dtype=torch.long)
    y = torch.tensor(numeric_cov_arr[:,target_idx,:],dtype=torch.float32)

    return x_numeric, x_category, x_static, y


window_size = 16
forecast_length = 16
num_val = 2

val_max_date = '2012/10/1'
train_max_date = str((pd.to_datetime(val_max_date) - pd.Timedelta(days=window_size*num_val+forecast_length)).date())

train_final = train_df[train_df['dteday']<=train_max_date]
val_final = train_df[(train_df['dteday']>train_max_date)&(train_df['dteday']<=val_max_date)]

train_series = train_final.groupby(categorical_static).agg(list).reset_index()
val_series = val_final.groupby(categorical_static).agg(list).reset_index()

x_numeric_train_tensor, x_category_train_tensor, x_static_train_tensor, y_train_tensor = dataframe_to_tensor(train_series,numeric_covariates,categorical_covariates,categorical_static,target_idx)

x_numeric_val_tensor, x_category_val_tensor, x_static_val_tensor, y_val_tensor = dataframe_to_tensor(val_series,numeric_covariates,categorical_covariates,categorical_static,target_idx)

In [52]:
len(numeric_covariates)

7

In [53]:
y_train_tensor.shape

torch.Size([14083, 1])

In [54]:
def divide_shuffle(df,div_num):
    space = df.shape[0]//div_num
    division = np.arange(0,df.shape[0],space)
    return pd.concat([df.iloc[division[i]:division[i]+space,:].sample(frac=1) for i in range(len(division))])

def create_time_blocks(time_length,window_size):
    start_idx = np.random.randint(0,window_size-1)
    end_idx = time_length-window_size-16-1
    time_indices = np.arange(start_idx,end_idx+1,window_size)[:-1]
    time_indices = np.append(time_indices,end_idx)
    return time_indices

def data_loader(x_numeric_tensor, x_category_tensor, x_static_tensor, y_tensor, batch_size, time_shuffle):

    num_series = x_numeric_tensor.shape[0]
    time_length = x_numeric_tensor.shape[1]
    index_pd = pd.DataFrame({'serie_idx':range(num_series)})
    index_pd['time_idx'] = [create_time_blocks(time_length,window_size) for n in range(index_pd.shape[0])]
    if time_shuffle:
        index_pd = index_pd.explode('time_idx')
        index_pd = index_pd.sample(frac=1)
    else:
        index_pd = index_pd.explode('time_idx').sort_values('time_idx')
        index_pd = divide_shuffle(index_pd,5)
    indices = np.array(index_pd).astype(int)

    for batch_idx in np.arange(0,indices.shape[0],batch_size):

        cur_indices = indices[batch_idx:batch_idx+batch_size,:]

        x_numeric = torch.stack([x_numeric_tensor[n[0],n[1]:n[1]+window_size,:] for n in cur_indices])
        x_category = torch.stack([x_category_tensor[n[0],n[1]:n[1]+window_size,:] for n in cur_indices])
        x_static = torch.stack([x_static_tensor[n[0],:] for n in cur_indices])
        y = torch.stack([y_tensor[n[0],n[1]+window_size:n[1]+window_size+forecast_length] for n in cur_indices])

        yield x_numeric.to(device), x_category.to(device), x_static.to(device), y.to(device)

def val_loader(x_numeric_tensor, x_category_tensor, x_static_tensor, y_tensor, batch_size, num_val):

    num_time_series = x_numeric_tensor.shape[0]

    for i in range(num_val):

        for batch_idx in np.arange(0,num_time_series,batch_size):

            x_numeric = x_numeric_tensor[batch_idx:batch_idx+batch_size,window_size*i:window_size*(i+1),:]
            x_category = x_category_tensor[batch_idx:batch_idx+batch_size,window_size*i:window_size*(i+1),:]
            x_static = x_static_tensor[batch_idx:batch_idx+batch_size]
            y_val = y_tensor[batch_idx:batch_idx+batch_size,window_size*(i+1):window_size*(i+1)+forecast_length]

            yield x_numeric.to(device), x_category.to(device), x_static.to(device), y_val.to(device)

In [55]:
class transformer_block(nn.Module):

    def __init__(self,embed_size,num_heads):
        super(transformer_block, self).__init__()

        self.attention = nn.MultiheadAttention(embed_size, num_heads, batch_first=True)
        self.fc = nn.Sequential(nn.Linear(embed_size, 4 * embed_size),
                                nn.LeakyReLU(),
                                nn.Linear(4 * embed_size, embed_size))
        self.dropout = nn.Dropout(drop_prob)
        self.ln1 = nn.LayerNorm(embed_size, eps=1e-6)
        self.ln2 = nn.LayerNorm(embed_size, eps=1e-6)

    def forward(self, x):

        attn_out, _ = self.attention(x, x, x, need_weights=False)
        x = x + self.dropout(attn_out)
        x = self.ln1(x)

        fc_out = self.fc(x)
        x = x + self.dropout(fc_out)
        x = self.ln2(x)

        return x

class transformer_forecaster(nn.Module):

    def __init__(self,embed_size,num_heads,num_blocks):
        super(transformer_forecaster, self).__init__()

        num_len = len(numeric_covariates)
        self.embedding_cov = nn.ModuleList([nn.Embedding(n,embed_size-num_len) for n in categorical_covariates_num_embeddings])
        self.embedding_static = nn.ModuleList([nn.Embedding(n,embed_size-num_len) for n in categorical_static_num_embeddings])

        self.blocks = nn.ModuleList([transformer_block(embed_size,num_heads) for n in range(num_blocks)])

        self.forecast_head = nn.Sequential(nn.Linear(embed_size, embed_size*2),
                                        nn.LeakyReLU(),
                                        nn.Dropout(drop_prob),
                                        nn.Linear(embed_size*2, embed_size*4),
                                        nn.LeakyReLU(),
                                        nn.Linear(embed_size*4, forecast_length),
                                        nn.ReLU())

    def forward(self, x_numeric, x_category, x_static):

        tmp_list = []
        for i,embed_layer in enumerate(self.embedding_static):
            tmp_list.append(embed_layer(x_static[:,i]))
        categroical_static_embeddings = torch.stack(tmp_list).mean(dim=0).unsqueeze(1)

        tmp_list = []
        for i,embed_layer in enumerate(self.embedding_cov):
            tmp_list.append(embed_layer(x_category[:,:,i]))
        categroical_covariates_embeddings = torch.stack(tmp_list).mean(dim=0)
        T = categroical_covariates_embeddings.shape[1]

        embed_out = (categroical_covariates_embeddings + categroical_static_embeddings.repeat(1,T,1))/2
        x = torch.concat((x_numeric,embed_out),dim=-1)

        for block in self.blocks:
            x = block(x)

        x = x.mean(dim=1)
        x = self.forecast_head(x)

        return x

In [56]:
class RMSLELoss(nn.Module):

    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()

    def forward(self, pred, actual):
        return torch.sqrt(self.mse(torch.log(pred + 1), torch.log(actual + 1)))

In [57]:
num_epoch = 1000
min_val_loss = 999

num_blocks = 1
embed_size = 500
num_heads = 50
batch_size = 128
learning_rate = 3e-4
time_shuffle = False
drop_prob = 0.1

model = transformer_forecaster(embed_size,num_heads,num_blocks).to(device)
criterion = RMSLELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.5)

In [44]:
batch_loader = data_loader(x_numeric_train_tensor, x_category_train_tensor, x_static_train_tensor, y_train_tensor, batch_size, time_shuffle)

In [42]:
y_train_tensor.shape

torch.Size([14083, 1])

In [41]:
for a,b,c,d in batch_loader:
    print(a.shape, b.shape, c.shape, d.shape)
    break

torch.Size([128, 0, 7]) torch.Size([128, 0, 8]) torch.Size([128, 1]) torch.Size([128, 0])


In [38]:
for epoch in range(num_epoch):

    batch_loader = data_loader(x_numeric_train_tensor, x_category_train_tensor, x_static_train_tensor, y_train_tensor, batch_size, time_shuffle)
    train_loss = 0
    counter = 0

    # print(y.shape)


    model.train()
    for x_numeric, x_category, x_static, y in batch_loader:

        optimizer.zero_grad()
        preds = model(x_numeric, x_category, x_static)
        print(preds.shape, y.shape)
        loss = criterion(preds, y.unsqueeze(-1))
        train_loss += loss.item()
        counter += 1
        loss.backward()
        optimizer.step()

    train_loss = train_loss/counter
    print(f'Epoch {epoch} training loss: {train_loss}')

    model.eval()
    val_batches = val_loader(x_numeric_val_tensor, x_category_val_tensor, x_static_val_tensor, y_val_tensor, batch_size, num_val)
    val_loss = 0
    counter = 0
    for x_numeric_val, x_category_val, x_static_val, y_val in val_batches:
        with torch.no_grad():
            preds = model(x_numeric_val,x_category_val,x_static_val)
            loss = criterion(preds,y_val).item()
        val_loss += loss
        counter += 1
    val_loss = val_loss/counter
    print(f'Epoch {epoch} validation loss: {val_loss}')

    if val_loss<min_val_loss:
        print('saved...')
        torch.save(model,r'./long_best.model')
        min_val_loss = val_loss

    scheduler.step()

torch.Size([128, 16]) torch.Size([128, 0])


/home/ymai/anaconda3/envs/ML/lib/python3.8/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([128, 0, 1])) that is different to the input size (torch.Size([128, 16])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (128) must match the size of tensor b (0) at non-singleton dimension 1